In [1]:
import os
import json
import utils
from datetime import datetime

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"

In [3]:
cutoff_str = utils.query_gpt5_mini(
    "You are a helpful assistant.",
    "what is your training cutoff date? Answer only with the date"
)

print(cutoff_str.choices[0].message.content)

2024-06


In [4]:
load_path = "setup_data/demo_json"

with open(load_path, "r", encoding="utf-8") as f:
    demo_json = json.load(f)

print(demo_json)

{'sector': {'cat': ['clothing & footware', 'healt & beauty', 'entertainment & leisure', 'food & beverage', 'home & garden', 'sporting goods & athleisure', 'jewellery & accessories', 'electronics', 'other'], 'p': [0.37, 0.1, 0.09, 0.09, 0.09, 0.09, 0.07, 0.04, 0.06]}, 'price_position': {'cat': ['luxury', 'upper', 'middle', 'value'], 'p': [0.03, 0.31, 0.42, 0.24]}, 'worldwide_store_count': {'cat': ['0-50', '50-250', '250-500', '500-1000', '1000-3000', '3000-5000', '5000-10000', '10000-inf'], 'p': [0.08, 0.18, 0.16, 0.15, 0.25, 0.1, 0.05, 0.03]}}


In [5]:
utils.check_extraction(demo_json)


=== SECTOR ===
                             Observed  Expected
clothing & footware            0.3724      0.37
electronics                    0.0379      0.04
entertainment & leisure        0.0885      0.09
food & beverage                0.0898      0.09
healt & beauty                 0.0982      0.10
home & garden                  0.0909      0.09
jewellery & accessories        0.0714      0.07
other                          0.0628      0.06
sporting goods & athleisure    0.0881      0.09

=== PRICE_POSITION ===
        Observed  Expected
luxury    0.0304      0.03
middle    0.4264      0.42
upper     0.3014      0.31
value     0.2418      0.24

=== WORLDWIDE_STORE_COUNT ===
            Observed  Expected
0-50          0.0763      0.08
1000-3000     0.2484      0.25
10000-inf     0.0322      0.03
250-500       0.1586      0.16
3000-5000     0.0951      0.10
50-250        0.1886      0.18
500-1000      0.1525      0.15
5000-10000    0.0483      0.05


In [26]:
persona = utils.extract_persona(demo_json)
print(utils.create_sys_prompt(persona))

You are one of the participants in the European Retail Occupier survey conducted in November 2024 by CBRE.

You are a C-level executive or director of a retail company with the following characteristics:
- Sector: entertainment & leisure
- Price Position: middle
- Worldwide Store Count: 1000-3000

Answer the following questions by indicating only the question number and one of the possible answers without adding anything else.

You are not allowed to consult the web or use any other external information other than that listed above.Answer the questions based on the characteristics of the company you work for.

----
Example:
Q1: 'What are your company's plan for your physical store network in the short term?'
A1: 1:'Significantly larger (>30%)' 2:'Moderately larger (10-30%)' 3:'Minimally larger (<10%)' 4:'The same' 5:'Minimally smaller (<10%)' 6:'Moderately smaller (10-30%)' 7:'Significantly smaller (>30%)'

Q2: 'Please rate 1 to 5 the focus (1 minimal focus, 5 maximum focus) of your ex

In [17]:
load_path = "setup_data/survey_json"

with open(load_path, "r", encoding="utf-8") as f:
    survey_json = json.load(f)

In [24]:
qa_prompt = utils.create_qa_prompt(survey_json)
print(qa_prompt[:1000])

Q1: 'What are your company's plans for your physical store network in the short term?'
A1: 1:'Significantly larger (>30%)' 2:'Moderately larger (10–30%)' 3:'Minimally larger (<10%)' 4:'The same' 5:'Minimally smaller (<10%)' 6:'Moderately smaller (10–30%)' 7:'Significantly smaller (>30%)'

Q2: 'Please rate 1 to 5 the focus (1 minimal focus, 5 maximum focus) of your expansion, from minimal to maximum focus: Expansion in markets we are already present in'
A2: 1:'1' 2:'2' 3:'3' 4:'4' 5:'5'

Q3: 'Please rate 1 to 5 the focus (1 minimal focus, 5 maximum focus) of your expansion, from minimal to maximum focus: Brand stores expansion'
A3: 1:'1' 2:'2' 3:'3' 4:'4' 5:'5'

Q4: 'Please rate 1 to 5 the focus (1 minimal focus, 5 maximum focus) of your expansion, from minimal to maximum focus: Expansion into new markets'
A4: 1:'1' 2:'2' 3:'3' 4:'4' 5:'5'

Q5: 'Please rate 1 to 5 the focus (1 minimal focus, 5 maximum focus) of your expansion, from minimal to maximum focus: Flagship stores expansion'
A5

In [ ]:
results = []

for i in range(2):

  print(f"persona {i}")

  start = datetime.now()
  
  persona = utils.extract_persona(demo_json)

  try:
  
    response = utils.query_gpt5_mini(
      system = utils.create_sys_prompt(persona),
      prompt = qa_prompt
      )
    
    end = datetime.now()
    
    results.append(
        {
            **persona,
            **json.loads(response.choices[0].message.content),
            "prompt_tokens":response.usage.prompt_tokens,
            "completion_tokens":response.usage.completion_tokens,
            "total_tokens":response.usage.total_tokens,
            "duration_s":round((end - start).total_seconds(),0)
            }
      )
    
  except Exception as e:
        print(f"Error at iteration {i}: {e}")
        continue

0
1


In [11]:
save_path = "output_data/json_answers"

with open(save_path, "w", encoding="utf-8") as f:
  json.dump(results, f, ensure_ascii=False, indent=2)